## 다중 분류 문제
- 난방 부하 단계를 예측해주세요!

- 예측할 값(y): Heat_Load (Very Low, Low, Medium, High, Very High)
- 평가: f1-macro
- data: train.csv, test.csv
- 제출 형식: result.csv파일을 아래와 같은 형식으로 제출
~~~
pred
Very Low
Low
High
...
Very High
~~~

### 답안 제출 참고
- pd.read_csv('result.csv') 로 제출 코드 확인

회귀인지 분류인지 구분이 가장 중요 이거 못맞추면 2번 답 빵점임

그리고 타겟이 문자인지 숫자인지도 매우 중요함!

In [ ]:
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/6_2/energy_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/6_2/energy_test.csv")



In [ ]:
#데이터 크기 확인
train.shape, test.shape

((537, 10), (231, 9))

In [ ]:
#train샘플확인
train.head(2)

,Compac,Surf_Area,Wall_Area,Roof,Height,Orient,Glaze_Area,Glaze_Distr,Cool_Load,Heat_Load
0,0.74,686.0,245.0,220.5,Short,South,0.25,3,14.72,Very Low
1,0.98,514.5,294.0,Small,Tall,South,0.40,2,33.94,High


In [ ]:
#test샘플확인

test.head(2)

,Compac,Surf_Area,Wall_Area,Roof,Height,Orient,Glaze_Area,Glaze_Distr,Cool_Load
0,0.64,784.0,343.0,220.5,Short,South,0.4,4,22.25
1,0.82,612.5,318.5,Large,Tall,North,0.4,3,32.43


In [ ]:
#type 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Compac       537 non-null    float64
 1   Surf_Area    537 non-null    float64
 2   Wall_Area    537 non-null    float64
 3   Roof         537 non-null    object 
 4   Height       537 non-null    object 
 5   Orient       537 non-null    object 
 6   Glaze_Area   537 non-null    float64
 7   Glaze_Distr  537 non-null    int64  
 8   Cool_Load    537 non-null    float64
 9   Heat_Load    537 non-null    object 
dtypes: float64(5), int64(1), object(4)
memory usage: 42.1+ KB


In [ ]:
#타겟값 확인
train['Roof'].value_counts()

,count
Roof,
220.5,257
Large,141
Medium,92
Small,47


In [ ]:
#기초통계 train(object)
train.describe(include='object')


,Roof,Height,Orient,Heat_Load
count,537,537,537,537
unique,4,2,4,5
top,220.5,Tall,South,Very Low
freq,257,280,145,142


In [ ]:
#기초통계 test(object)
#위 트레인의 unique와 test 데이터의 uqnique의 갯수가 같으면 됨, 다르면 머신러닝이 안됨!!!
# 위에 Head-t_Load는 test 데이터에 없지만, 나중에 train에서 빼줄것이므로 괜찮음
test.describe(include='object')


,Roof,Height,Orient
count,231,231,231
unique,4,2,4
top,220.5,Short,North
freq,127,127,74


In [ ]:
#결측치 확인(train)
train.isnull().sum()

,0
Compac,0
Surf_Area,0
Wall_Area,0
Roof,0
Height,0
Orient,0
Glaze_Area,0
Glaze_Distr,0
Cool_Load,0
Heat_Load,0


In [ ]:
#결측치 확인(test)
test.isnull().sum()

,0
Compac,0
Surf_Area,0
Wall_Area,0
Roof,0
Height,0
Orient,0
Glaze_Area,0
Glaze_Distr,0
Cool_Load,0


In [ ]:
#target 확인
train['Heat_Load'].value_counts()

,count
Heat_Load,
Very Low,142
Low,123
High,122
Very High,79
Medium,71


####데이터 전처리

In [ ]:
#target 컬럼 처리
#처리하는 이유는 위에 unique의 값이 다르고 따로 보관해서 target값을 pred로 예측할때 쓰려고?
target = train.pop('Heat_Load')


In [ ]:
# 원 핫 인코딩(판다스)
#인코딩시 전후 크기를 비교하면 좋음
print(train.shape , test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape , test.shape)



(537, 16) (231, 16)
(537, 16) (231, 16)


검증데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train,
                                            target,
                                            test_size=0.2,
                                            random_state=0)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((429, 16), (108, 16), (429,), (108,))

머신러닝 학습 및 평가

In [ ]:
# 평가함수
from sklearn.metrics import f1_score

In [ ]:
#랜덤포레스트
#macro라는건 처음봄
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)


f1_score(y_val, pred, average='macro')

0.9277616846430405

In [ ]:
#값위에 막 이상한거 나오면 verbose=-1써주면 됨
import lightgbm as lgb
lg = lgb.LGBMClassifier(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)

f1_score(y_val, pred, average='macro')

0.9319703995747778

예측 및 결과 파일 생성


In [ ]:
#test 예측
pred = rf.predict(test)
submit = pd.DataFrame({
            'pred' : pred
})

submit.to_csv('result.csv', index=False)

In [ ]:
pd.read_csv('result.csv')

,pred
0,Low
1,High
2,High
3,Low
4,Low
...,...
226,Very Low
227,Medium
228,Very Low
229,Low


##심화학습 크로스 벨리데이션!
###크로스 벨리데이션은 학습이 되거나 예측이 되는게 아니라 검증용임!

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/6_2/energy_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/6_2/energy_test.csv")

# 데이터 전처리
target = train.pop('Heat_Load')
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 모델 준비
rf = RandomForestClassifier(random_state=0)

# 크로스 밸리데이션 수행(5-fold)
from sklearn.model_selection import cross_val_score
result = cross_val_score(rf, train, target, cv=5, scoring='f1_macro')

# 평균 F1 스코어 출력
result.mean()

In [ ]:
# 최종 제출 파일
rf.fit(train, target)
pred = rf.predict(test)
submit = pd.DataFrame({'pred': pred})
submit.to_csv("result.csv", index=False)

In [ ]:
# 시험환경에서 scorers 확인 (코랩X)
# from sklearn.metrics import SCORERS

# available_scorers = sorted(SCORERS.keys())
# available_scorers

In [ ]:
# 코랩에서 scorers 확인 (시험환경X)
from sklearn.metrics import get_scorer_names

available_scorers = sorted(get_scorer_names())
print(available_scorers)